In [8]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from inference.infer import PoseDetector, ActionClassifier

In [2]:
# list 타입인 file 변수 중 원하는 영상에 접근 하시면 됩니다. 
# list 슬라이싱 사용하시면 되여
# -------------------------------------------------------------------
os.chdir('../')
files = os.listdir('data_folder/src/')
files

['good.mp4',
 'KakaoTalk_20221110_160630579.mp4',
 'KakaoTalk_20221110_160630635.mp4',
 'KakaoTalk_20221110_160633025.mp4',
 'KakaoTalk_20221110_160635468.mp4',
 'KakaoTalk_20221112_184323826_yg.mp4',
 'label.mp4',
 'mac.mp4',
 'random.mp4',
 'sm.mp4',
 'sm2.mp4',
 'sm3.mp4',
 '가볍게 치는 좋은 스윙 슬로우모션(GOLF SWING KLPGA).mp4']

In [3]:
file = files[1] # 이부분 변경하시면 되여 예) files[0] -> files[1]
src_dir = os.path.join('data_folder/src/', file)
src_dir

'data_folder/src/KakaoTalk_20221110_160630579.mp4'

In [4]:
# 영상을 읽어 address, top 동작 등에 맞는 프레임을 찾아 
# 좌표들을 반환하는 코드입니다.
# ----------------------------------------------------------
pose = PoseDetector()
ac = ActionClassifier()
with open('data_folder/parameter/params.json', 'r') as r:
    params = json.load(r)
actions = params.keys()
for action in actions:
    video = cv2.VideoCapture(src_dir)
    while video.isOpened():
        read_ok, frame = video.read()
        if not read_ok:
            print('TRK Complete!')
            break
        cnt = video.get(cv2.CAP_PROP_POS_FRAMES)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, dsize=(0, 0), fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
        pose_obj = pose.trk_process(image=frame)
        pose.get_coordis_3d(pose_obj)
        landmarked_img = pose.drawing_with_pose(frame, pose_obj)
        ac.update_information(action=action,
                            parameter=params,
                            pose_coordis=pose.coordinates,
                            num_frame=cnt, image=landmarked_img)

TRK Complete!
TRK Complete!
TRK Complete!
TRK Complete!
TRK Complete!


#### 데이터 설명 
 - 위 셀이 실행 되면 ac.esti_inform 속성으로 데이터가 저장됩니다.
 - ac.esti_inform의 데이터 구성은 아래와 같습니다.
 - {동작 구분 : {닮음 정도, 좌표, 이미지}}

In [5]:
# address 동작의 좌표들에 접근하기 위해서는 아래 코드와 같이 접근합니다.
# ac.esti_inform['address']['coordinate'] -> dictionary 타입
# ------------------------------------------------------------------
ac.esti_inform['address']['coordinate']

{'nose': array([-0.03106143, -0.57431102, -0.44317898]),
 'left_eye_inner': array([-0.03219242, -0.61335385, -0.42692801]),
 'left_eye': array([-0.03186603, -0.61368579, -0.42623177]),
 'left_eye_outer': array([-0.03188009, -0.61425096, -0.42587665]),
 'right_eye_inner': array([-0.0630623 , -0.60405284, -0.42792735]),
 'right_eye': array([-0.06262047, -0.60522789, -0.42858401]),
 'right_eye_outer': array([-0.0629364 , -0.60647959, -0.42770159]),
 'left_ear': array([ 0.01549635, -0.60056865, -0.31570807]),
 'right_ear': array([-0.12225794, -0.57241392, -0.32777762]),
 'mouth_left': array([-0.00467491, -0.55020648, -0.40457746]),
 'mouth_right': array([-0.04584594, -0.53966343, -0.4069823 ]),
 'left_shoulder': array([ 0.13043192, -0.43318024, -0.23748171]),
 'right_shoulder': array([-0.17407763, -0.40052697, -0.25677893]),
 'left_elbow': array([ 0.10947999, -0.20597143, -0.26692522]),
 'right_elbow': array([-0.18471038, -0.15243277, -0.22847749]),
 'left_wrist': array([ 0.03620585,  0.00

In [7]:
# 위 데이터는 각 포인트별로 엉덩이 가운데를 기준으로 얼마나 떨어져 있는지를 x, y, z
# 좌표값으로 줍니다
# 예시) ac.esti_inform['address']['coordinate']['nose']  
#              └> 어드레스 동작중 nose의 x, y, z 값(np.ndarray 형태)  
# ---------------------------------------------------------------------------------
ac.esti_inform['backswing']['coordinate']['left_wrist']

array([-0.39508978, -0.59152544, -0.35853028])